<h1><center>Capstone Project - The Battle of Neighborhoods</h1></center>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>


Toronto is one of the diverse and popular cities in Canada. It is also being recognized as one of the financial capitals of Canada. There are a number of restaurants available in Toronto due to the cultural diversity which leads people trying out different restaurants and testing there cuisines. It is difficult to find the best location to open up a new restaurant in the city of Toronto as there are lots of restaurants. <br>

The goal of this project is to help finding the best location to open up a new restaurant in order to gain maximum profit and engagement. <br>

In short, the problem  is <b>“which is a good location in Toronto to open up a new restaurant?”</b><br>

There are several factors need to be considered to choose the best place and I will data scienc epowers to find promising neighorhoods. This project gives stakeholders a brief idea for opening new restaurants in the city of Toronto.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* other fadcilities such as park, university, hotel etc.

The frist step will be gathing data from differeent source;

<h5> Import libraries </h5>

###### Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Download and Explore Dataset

In [5]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Let's rename the columns

In [6]:
df.rename(columns={'Neighbourhood':'Neighborhood', 'Postcode': 'PostalCode'},inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Let's ignore the rows where Borough and Neighborhood are not assigned

In [7]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


We need the latitude and longitude of these areas. Let's download this dataset directly.

In [8]:
geo=pd.read_csv("https://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename the Postal Code column to match with the other dataset!

In [10]:
geo.rename(columns={'Postal Code': 'PostalCode'},inplace=True)
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to combine these two datasets!

In [11]:
df_neigh = df.merge(geo,left_on=['PostalCode'],right_on=['PostalCode'], how='inner')
df_neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


Let's slice the original dataframe and create a new dataframe of the Toronto data

In [12]:
toronto_df = df_neigh[df_neigh['Borough'].str.contains('Toronto', na = False)]
toronto_df.reset_index(drop = True, inplace = True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


##### Explore the neighborhoods in Toronto

In order to define an instance of the geocoder, we need to define a user_agent. I will name the agent my_explorer, as shown below.

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [102]:
# Creating a map of Toronto.

lat = 43.653963
long = -79.387207

map_toronto = folium.Map(location=[lat, long], zoom_start=11)

for lat, long, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'IWJSUY3UUD2DYZ43JXTA0TNY45QTJNEJULSG3Y5BRK3MAAJU' # your Foursquare ID
CLIENT_SECRET = 'UXUPZGLXTP2WD0KZ1NL5TCZQEHCPMVYDVY3FXDXJBXS4VEYU' # your Foursquare Secret
VERSION = '20191107' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 1000
LIMIT = 200

Your credentails:
CLIENT_ID: IWJSUY3UUD2DYZ43JXTA0TNY45QTJNEJULSG3Y5BRK3MAAJU
CLIENT_SECRET:UXUPZGLXTP2WD0KZ1NL5TCZQEHCPMVYDVY3FXDXJBXS4VEYU


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's latitude and longitude values.

In [22]:

neighbourhood_latitude = toronto_df.loc[0,'Latitude']
neighbourhood_longitude = toronto_df.loc[0,'Longitude']
neighbourhood_name = toronto_df.loc[0,'Neighborhood']

print('the coordinates of {} are {} and {}'.format(neighbourhood_name,neighbourhood_latitude,neighbourhood_longitude))

the coordinates of Harbourfront are 43.6542599 and -79.3606359


Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.

In [23]:
LIMIT = 100
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                            CLIENT_SECRET,
                                                                                                                            VERSION,
                                                                                                                            neighbourhood_latitude,
                                                                                                                            neighbourhood_longitude,
                                                                                                                           radius,
                                                                                                                           LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IWJSUY3UUD2DYZ43JXTA0TNY45QTJNEJULSG3Y5BRK3MAAJU&client_secret=UXUPZGLXTP2WD0KZ1NL5TCZQEHCPMVYDVY3FXDXJBXS4VEYU&v=20191107&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [24]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5deff27769babe001bce3540'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


##### Explore Neighborhoods in Toronto

Let's get the top 200 venues in Toronto that are within 1000 m

In [62]:

#getting nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [63]:

toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

In [64]:
print(toronto_venues.shape)
toronto_venues.head()

(5755, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Harbourfront,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


Let's create a dataset called toronto_restaurants where type is Restaurant

In [65]:
toronto_venues['type'] = toronto_venues['Venue Category'].apply(lambda x:x.split()[-1])
toronto_restaurants = toronto_venues[toronto_venues['type'] == 'Restaurant'].reset_index(drop=True)

In [66]:
toronto_restaurants.drop(['type'],axis=1,inplace=True)
toronto_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
1,Harbourfront,43.654260,-79.360636,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
2,Harbourfront,43.654260,-79.360636,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
3,Harbourfront,43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,Harbourfront,43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant
5,Harbourfront,43.654260,-79.360636,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
6,Harbourfront,43.654260,-79.360636,Sukhothai,43.658444,-79.365681,Thai Restaurant
7,Harbourfront,43.654260,-79.360636,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
8,Harbourfront,43.654260,-79.360636,Fusaro's,43.653347,-79.369517,Italian Restaurant
9,Harbourfront,43.654260,-79.360636,Archeo,43.650667,-79.359431,Italian Restaurant


In [67]:
toronto_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
1,Harbourfront,43.65426,-79.360636,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
2,Harbourfront,43.65426,-79.360636,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
3,Harbourfront,43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,Harbourfront,43.65426,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant


## Methodology <a name="methodology"></a>

In this project we will detect areas of Toronot that have low restaurant density and have other amenities nearby.

In the frist step we already colletcted the required data of every restuarant and identified their category. 

In the second step we will calculate and explore the restaurant density across different areas of Toronto and identify the areas with low restaurant density. 

In the final step we will focus on the  most promising areas which also require some other basic requirements. 


## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data.

Let's visualize the restuarants in Toronto

In [68]:

map_toronto_restaurants = folium.Map(location=[latitude,longitude],zoom_start=12)

for lat,lng,label in zip(toronto_restaurants['Venue Latitude'], toronto_restaurants['Venue Longitude'], toronto_restaurants['Venue']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto_restaurants)
    
map_toronto_restaurants

Let's see the number of restuarnats in each category

In [70]:
toronto_restaurants['Venue Category'].value_counts()

Italian Restaurant                 166
Restaurant                         163
Sushi Restaurant                   111
Japanese Restaurant                 95
Vegetarian / Vegan Restaurant       86
Thai Restaurant                     79
American Restaurant                 60
French Restaurant                   49
Mexican Restaurant                  46
Fast Food Restaurant                40
Indian Restaurant                   39
Greek Restaurant                    38
Vietnamese Restaurant               36
Ramen Restaurant                    31
Asian Restaurant                    31
Seafood Restaurant                  30
Middle Eastern Restaurant           25
New American Restaurant             24
Mediterranean Restaurant            24
Caribbean Restaurant                24
Chinese Restaurant                  21
Korean Restaurant                   18
Latin American Restaurant           16
Comfort Food Restaurant             15
Tapas Restaurant                    15
Modern European Restauran

##### Analyze Each Neighborhood

In [71]:
# Convert each category to one hot
toronto_one_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Add neighborhood back
toronto_one_hot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
toronto_one_hot = toronto_one_hot.set_index('Neighborhood').reset_index()

toronto_one_hot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Now group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [72]:
toronto_grouped = toronto_one_hot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,Adelaide,0.00,0.0,0.000000,0.000000,0.020000,0.0,0.0,0.0,0.0,0.0,0.01,0.0100

Let's print each neighborhood along with the top 5 most common venues

In [74]:

num_top_restaurants = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_restaurants))
    print('\n')

----Adelaide----
              venue  freq
0              Café  0.08
1             Hotel  0.05
2       Coffee Shop  0.05
3           Theater  0.04
4  Sushi Restaurant  0.03


----Bathurst Quay----
              venue  freq
0       Coffee Shop  0.13
1   Harbor / Marina  0.13
2    Scenic Lookout  0.07
3              Park  0.07
4  Sculpture Garden  0.07


----Berczy Park----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2        Hotel  0.05
3     Beer Bar  0.04
4   Restaurant  0.04


----Brockton----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.06
2              Restaurant  0.05
3  Furniture / Home Store  0.04
4                     Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.08
2         Pizza Place  0.06
3             Brewery  0.06
4  Italian Restaurant  0.04


----CN Tower----
              venue  freq
0       Coffee 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Hotel,Coffee Shop,Theater,Sushi Restaurant,Gastropub,Asian Restaurant,Japanese Restaurant,Steakhouse,Beer Bar
1,Bathurst Quay,Coffee Shop,Harbor / Marina,Tunnel,Scenic Lookout,Sculpture Garden,Dog Run,Track,Park,Dance Studio,Café
2,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Beer Bar,Restaurant,Bakery,Farmers Market,Steakhouse,Seafood Restaurant
3,Brockton,Café,Coffee Shop,Restaurant,Bakery,Furniture / Home Store,Bar,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Park
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Park,Brewery,Pizza Place,Sushi Restaurant,Pet Store,Italian Restaurant,Diner,Snack Place,Beach


Let's create a dataframe called toronto_amenities to cluster

In [80]:
toronto_amenities = toronto_grouped[["Neighborhood","Restaurant", 'Hotel', 'Movie Theater', 'Park', 'University']]
toronto_amenities.head()

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University
0,Adelaide,0.020000,0.05,0.02,0.000000,0.01
1,Bathurst Quay,0.000000,0.00,0.00,0.066667,0.00
2,Berczy Park,0.040000,0.05,0.01,0.020000,0.00
3,Brockton,0.050000,0.02,0.00,0.020000,0.00
4,Business Reply Mail Processing Centre 969 Eastern,0.020833,0.00,0.00,0.083333,0.00


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [82]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_amenities.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 2, 3, 4, 4, 1, 0, 0, 0])

In [84]:

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_amenities
toronto_merged["Cluster Labels"]=kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels
0,Adelaide,0.020000,0.05,0.02,0.000000,0.01,2
1,Bathurst Quay,0.000000,0.00,0.00,0.066667,0.00,4
2,Berczy Park,0.040000,0.05,0.01,0.020000,0.00,2
3,Brockton,0.050000,0.02,0.00,0.020000,0.00,3
4,Business Reply Mail Processing Centre 969 Eastern,0.020833,0.00,0.00,0.083333,0.00,4


In [88]:
toronto_merged = toronto_merged.join(toronto_df.set_index("Neighborhood"), on="Neighborhood")

print(toronto_merged.shape)
toronto_merged.head()

(74, 11)


,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,Adelaide,0.020000,0.05,0.02,0.000000,0.01,2,M5H,Downtown Toronto,43.650571,-79.384568
1,Bathurst Quay,0.000000,0.00,0.00,0.066667,0.00,4,M5V,Downtown Toronto,43.628947,-79.394420
2,Berczy Park,0.040000,0.05,0.01,0.020000,0.00,2,M5E,Downtown Toronto,43.644771,-79.373306
3,Brockton,0.050000,0.02,0.00,0.020000,0.00,3,M6K,West Toronto,43.636847,-79.428191
4,Business Reply Mail Processing Centre 969 Eastern,0.020833,0.00,0.00,0.083333,0.00,4,M7Y,East Toronto,43.662744,-79.321558


Finally, let's visualize the resulting clusters

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###### Examine Clusters

In [94]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
7,Central Bay Street,0.010000,0.01,0.01,0.030000,0.01,0,M5G,Downtown Toronto,43.657952,-79.387383
8,Chinatown,0.010000,0.00,0.00,0.000000,0.00,0,M5T,Downtown Toronto,43.653206,-79.400049
9,Christie,0.010000,0.00,0.00,0.020000,0.00,0,M6G,Downtown Toronto,43.669542,-79.422564
12,Davisville,0.020000,0.00,0.02,0.010000,0.00,0,M4S,Central Toronto,43.704324,-79.388790
13,Davisville North,0.020000,0.01,0.02,0.010000,0.00,0,M4P,Central Toronto,43.712751,-79.390197
24,Grange Park,0.010000,0.00,0.00,0.000000,0.00,0,M5T,Downtown Toronto,43.653206,-79.400049
29,High Park,0.020000,0.00,0.00,0.030000,0.00,0,M6P,West Toronto,43.661608,-79.464763
32,Kensington Market,0.010000,0.00,0.00,0.000000,0.00,0,M5T,Downtown Toronto,43.653206,-79.400049
40,Parkdale,0.030000,0.00,0.01,0.010000,0.00,0,M6R,West Toronto,43.648960,-79.456325
46,Riverdale,0.030000,0.00,0.00,0.010000,0.00,0,M4K,East Toronto,43.679557,-79.352188


In [95]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
6,Cabbagetown,0.027778,0.000000,0.000000,0.055556,0.0,1,M4X,Downtown Toronto,43.667967,-79.367675
10,Church and Wellesley,0.020000,0.020000,0.000000,0.040000,0.0,1,M4Y,Downtown Toronto,43.665860,-79.383160
14,Deer Park,0.025974,0.012987,0.012987,0.064935,0.0,1,M4V,Central Toronto,43.686412,-79.400049
21,Forest Hill SE,0.025974,0.012987,0.012987,0.064935,0.0,1,M4V,Central Toronto,43.686412,-79.400049
26,Harbourfront,0.040000,0.000000,0.000000,0.040000,0.0,1,M5A,Downtown Toronto,43.654260,-79.360636
30,India Bazaar,0.025000,0.000000,0.012500,0.050000,0.0,1,M4L,East Toronto,43.668999,-79.315572
37,Moore Park,0.033898,0.000000,0.016949,0.050847,0.0,1,M4T,Central Toronto,43.689574,-79.383160
39,North Toronto West,0.022727,0.000000,0.000000,0.068182,0.0,1,M4R,Central Toronto,43.715383,-79.405678
44,Rathnelly,0.025974,0.012987,0.012987,0.064935,0.0,1,M4V,Central Toronto,43.686412,-79.400049
52,South Hill,0.025974,0.012987,0.012987,0.064935,0.0,1,M4V,Central Toronto,43.686412,-79.400049


In [96]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,Adelaide,0.020000,0.050000,0.020000,0.000000,0.01,2,M5H,Downtown Toronto,43.650571,-79.384568
2,Berczy Park,0.040000,0.050000,0.010000,0.020000,0.00,2,M5E,Downtown Toronto,43.644771,-79.373306
11,Commerce Court,0.030000,0.050000,0.000000,0.010000,0.00,2,M5L,Downtown Toronto,43.648198,-79.379817
15,Design Exchange,0.020000,0.070000,0.010000,0.020000,0.00,2,M5K,Downtown Toronto,43.647177,-79.381576
19,First Canadian Place,0.030000,0.070000,0.010000,0.010000,0.00,2,M5X,Downtown Toronto,43.648429,-79.382280
27,Harbourfront East,0.050000,0.060000,0.010000,0.030000,0.00,2,M5J,Downtown Toronto,43.640816,-79.381752
33,King,0.020000,0.050000,0.020000,0.000000,0.01,2,M5H,Downtown Toronto,43.650571,-79.384568
45,Richmond,0.020000,0.050000,0.020000,0.000000,0.01,2,M5H,Downtown Toronto,43.650571,-79.384568
54,St. James Town,0.036765,0.036765,0.007353,0.029412,0.00,2,M5C,Downtown Toronto,43.651494,-79.375418
54,St. James Town,0.036765,0.036765,0.007353,0.029412,0.00,2,M4X,Downtown Toronto,43.667967,-79.367675


In [98]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
3,Brockton,0.05,0.02,0.00,0.02,0.0,3,M6K,West Toronto,43.636847,-79.428191
18,Exhibition Place,0.05,0.02,0.00,0.02,0.0,3,M6K,West Toronto,43.636847,-79.428191
23,Garden District,0.03,0.02,0.01,0.02,0.0,3,M5B,Downtown Toronto,43.657162,-79.378937
25,Harbord,0.04,0.01,0.00,0.02,0.0,3,M5S,Downtown Toronto,43.662696,-79.400049
36,Little Portugal,0.05,0.01,0.00,0.02,0.0,3,M6J,West Toronto,43.647927,-79.419750
38,North Midtown,0.04,0.01,0.00,0.01,0.0,3,M5R,Central Toronto,43.672710,-79.405678
41,Parkdale Village,0.05,0.02,0.00,0.02,0.0,3,M6K,West Toronto,43.636847,-79.428191
51,Ryerson,0.03,0.02,0.01,0.02,0.0,3,M5B,Downtown Toronto,43.657162,-79.378937
60,The Annex,0.04,0.01,0.00,0.01,0.0,3,M5R,Central Toronto,43.672710,-79.405678
67,Trinity,0.05,0.01,0.00,0.02,0.0,3,M6J,West Toronto,43.647927,-79.419750


In [99]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Neighborhood,Restaurant,Hotel,Movie Theater,Park,University,Cluster Labels,PostalCode,Borough,Latitude,Longitude
1,Bathurst Quay,0.000000,0.0,0.0,0.066667,0.0,4,M5V,Downtown Toronto,43.628947,-79.394420
4,Business Reply Mail Processing Centre 969 Eastern,0.020833,0.0,0.0,0.083333,0.0,4,M7Y,East Toronto,43.662744,-79.321558
5,CN Tower,0.000000,0.0,0.0,0.066667,0.0,4,M5V,Downtown Toronto,43.628947,-79.394420
16,Dovercourt Village,0.014493,0.0,0.0,0.086957,0.0,4,M6H,West Toronto,43.669005,-79.442259
17,Dufferin,0.014493,0.0,0.0,0.086957,0.0,4,M6H,West Toronto,43.669005,-79.442259
20,Forest Hill North,0.000000,0.0,0.0,0.086957,0.0,4,M5P,Central Toronto,43.696948,-79.411307
22,Forest Hill West,0.000000,0.0,0.0,0.086957,0.0,4,M5P,Central Toronto,43.696948,-79.411307
28,Harbourfront West,0.000000,0.0,0.0,0.066667,0.0,4,M5V,Downtown Toronto,43.628947,-79.394420
31,Island airport,0.000000,0.0,0.0,0.066667,0.0,4,M5V,Downtown Toronto,43.628947,-79.394420
34,King and Spadina,0.000000,0.0,0.0,0.066667,0.0,4,M5V,Downtown Toronto,43.628947,-79.394420


## Results and Discussion <a name="results"></a>

The analysis shows that although there is a great number of restaurants in Toronto, there are areas of low restaurant density as well as other amenities like Movie theatre, Universities, Parks etc. Highest concentration of restaurants was detected on cluster 4 and lowest concentration was detected on cluster 5. 

Final decision on finding the best location will be made by stakeholders based on specific characteristics of neighborhoods, and locations in every recommended zone, taking into consideration additional factors like rental prices, security, and other an attractiveness.


## Conclusion

This Project helps to better understand the venues in that neighborhood. After analyzing all the neighborhoods, I found that cluster 3 has a smaller number of restaurants and thus it would be best to consider open a restaurant there. For better understanding, other factors such as crime rates, public transport facilities, rental price should be added into consideration. The final decision on finding the best location will be made by Stakeholders.